In [ ]:
!pip install tensorflow --upgrade


In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

import seaborn as sns
import joblib
import matplotlib.pyplot as plt

In [2]:
#we are gonna do logistic regression, xgboost, a neural network, xgboost, and random forest

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(train_data.head())
print(train_data.info())
print(test_data.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Preprocess the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())

le_sex = LabelEncoder()
le_embarked = LabelEncoder()

train_data['Sex'] = le_sex.fit_transform(train_data['Sex'])
train_data['Embarked'] = le_embarked.fit_transform(train_data['Embarked'])

joblib.dump(le_sex, 'le_sex.pkl')
joblib.dump(le_embarked, 'le_embarked.pkl')

train_data = train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
# Logistic Regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)
log_reg_preds = log_reg_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, log_reg_preds))

Logistic Regression Accuracy: 0.8044692737430168


In [6]:
# XGBoost
xgb_model = XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))

XGBoost Accuracy: 0.7877094972067039


In [26]:
# Neural Network
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

nn_model = Sequential([
    Dense(32, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)

nn_loss, nn_accuracy = nn_model.evaluate(X_test, y_test_cat)
print("Neural Network Accuracy:", nn_accuracy)

joblib.dump(log_reg_model, 'log_reg_model.pkl')
joblib.dump(xgb_model, 'xgb_model.pkl')
nn_model.save('nn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.5394 - loss: 0.7524 - val_accuracy: 0.7343 - val_loss: 0.6407
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6400 - loss: 0.6541 - val_accuracy: 0.7902 - val_loss: 0.5886
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6925 - loss: 0.6302 - val_accuracy: 0.7972 - val_loss: 0.5520
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7273 - loss: 0.5765 - val_accuracy: 0.7762 - val_loss: 0.5233
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7388 - loss: 0.5743 - val_accuracy: 0.7972 - val_loss: 0.4953
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7485 - loss: 0.5577 - val_accuracy: 0.7972 - val_loss: 0.4748
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7553 - loss: 0.5518 - val_accuracy: 0.8042 - val_loss: 0.4587
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7356 - loss: 0.5501 - val_accuracy: 0.8112 - val_

Neural Network Accuracy: 0.7444444298744202


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict(X_test)
print("K-Nearest Neighbors Accuracy:", accuracy_score(y_test, knn_preds))

joblib.dump(knn_model, 'knn_model.pkl')

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))

joblib.dump(rf_model, 'rf_model.pkl')

K-Nearest Neighbors Accuracy: 0.8044692737430168
Random Forest Accuracy: 0.8212290502793296


['rf_model.pkl']

In [28]:
def ensemble_predict(xgb_model, log_reg_model, nn_model, knn_model, rf_model, input_features, weights=[1, 1, 1, 3, 3]):
    """
    Function to ensemble predictions by averaging the predicted probabilities of multiple models, giving more weight to the more complex models.
    """
    xgb_prob = xgb_model.predict_proba(input_features)[:, 1][0]
    log_reg_prob = log_reg_model.predict_proba(input_features)[:, 1][0]
    nn_prob = nn_model.predict(input_features)[0][1]
    knn_prob = knn_model.predict_proba(input_features)[:, 1][0]
    rf_prob = rf_model.predict_proba(input_features)[:, 1][0]

    total_weight = sum(weights)
    weighted_avg_prob = (
        xgb_prob * weights[0] +
        log_reg_prob * weights[1] +
        nn_prob * weights[2] +
        knn_prob * weights[3] +
        rf_prob * weights[4]
    ) / total_weight

    return weighted_avg_prob

def predict_survival(xgb_model, log_reg_model, nn_model, knn_model, rf_model):
    print("Enter your details to predict survival:")
    pclass = int(input("Passenger Class (1 = 1st, 2 = 2nd, 3 = 3rd): "))
    sex = input("Sex (male/female): ").strip().lower()
    age = float(input("Age: "))
    sibsp = int(input("Number of Siblings/Spouses Aboard: "))
    parch = int(input("Number of Parents/Children Aboard: "))
    fare = float(input("Ticket Fare: "))
    embarked = input("Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton): ").strip().upper()

    sex_encoded = le_sex.transform([sex])[0]
    embarked_encoded = le_embarked.transform([embarked])[0]
    input_features = np.array([[pclass, sex_encoded, age, sibsp, parch, fare, embarked_encoded]])

    xgb_prob = xgb_model.predict_proba(input_features)[:, 1][0]
    log_reg_prob = log_reg_model.predict_proba(input_features)[:, 1][0]
    nn_prob = nn_model.predict(input_features)[0][1]
    knn_prob = knn_model.predict_proba(input_features)[:, 1][0]
    rf_prob = rf_model.predict_proba(input_features)[:, 1][0]

    final_prob = ensemble_predict(xgb_model, log_reg_model, nn_model, knn_model, rf_model, input_features)

    print(f"XGBoost Model Probability: {xgb_prob * 100:.2f}%")
    print(f"Logistic Regression Model Probability: {log_reg_prob * 100:.2f}%")
    print(f"Neural Network Model Probability: {nn_prob * 100:.2f}%")
    print(f"K-Nearest Neighbors Model Probability: {knn_prob * 100:.2f}%")
    print(f"Random Forest Model Probability: {rf_prob * 100:.2f}%")
    print(f"Final Survival Probability: {final_prob * 100:.2f}%")

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

log_reg_model = joblib.load('log_reg_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')
nn_model = tf.keras.models.load_model('nn_model.h5')
knn_model = joblib.load('knn_model.pkl')
rf_model = joblib.load('rf_model.pkl')

def print_metrics(y_test, predictions, model_name):
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

log_reg_preds = log_reg_model.predict(X_test)
print_metrics(y_test, log_reg_preds, "Logistic Regression")

xgb_preds = xgb_model.predict(X_test)
print_metrics(y_test, xgb_preds, "XGBoost")

nn_preds = np.argmax(nn_model.predict(X_test), axis=-1)
print_metrics(y_test, nn_preds, "Neural Network")

knn_preds = knn_model.predict(X_test)
print_metrics(y_test, knn_preds, "K-Nearest Neighbors")

rf_preds = rf_model.predict(X_test)
print_metrics(y_test, rf_preds, "Random Forest")

predict_survival(xgb_model, log_reg_model, nn_model, knn_model, rf_model)

Logistic Regression - Accuracy: 0.7333, Precision: 0.6471, Recall: 0.6471, F1 Score: 0.6471
XGBoost - Accuracy: 0.7667, Precision: 0.6757, Recall: 0.7353, F1 Score: 0.7042
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Neural Network - Accuracy: 0.7444, Precision: 0.6571, Recall: 0.6765, F1 Score: 0.6667
K-Nearest Neighbors - Accuracy: 0.7444, Precision: 0.6774, Recall: 0.6176, F1 Score: 0.6462
Random Forest - Accuracy: 0.7778, Precision: 0.7333, Recall: 0.6471, F1 Score: 0.6875
Enter your details to predict survival:
Passenger Class (1 = 1st, 2 = 2nd, 3 = 3rd): 1
Sex (male/female): female
Age: 12
Number of Siblings/Spouses Aboard: 1
Number of Parents/Children Aboard: 1
Ticket Fare: 20
Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton): S
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
XGBoost Model Probability: 50.11%
Logistic Regression Model Probability: 1.13%
Neural Network Model Probability: 77.79%
K-Nearest Neighbors Model Probability: 80.00%
Random Forest Model Probability: 25.67%
Final Survival Probability: 49.56%
